In [1]:
import numpy as np
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from IPython.display import clear_output

import h5py
import time
import os

In [29]:
outputname = "./taco_imgFeat_vgg16.h5"
imgDone = {}

try:
    f_out = h5py.File(outputname, "r")

    for k in list(f_out.keys()):
        imgDone[k] = 1

    f_out.close()
except:
    pass

len(imgDone)

1500

In [18]:
# load pretrained model
model = VGG16(weights="imagenet", include_top=False)

In [19]:
from keras.layers import Dense

In [20]:
def getEstimatedTime(done, doneTime, pending):
    if done != 0:
        return (doneTime*pending)/done
    else:
        return 0

def progressBar(perc, startTime):
    clear_output(wait=True)
    print("[",end="")
    for j in range(30):
        if perc > (j+2)*int(100/30):
            print("=",end="")
        elif perc > (j+1)*int(100/30):
            print(">",end="")
        else:
            print(".",end="")
    print("] "+str(round(perc,2))+"%\t",end="")
    doneTime = time.time() - startTime
    print("Est Time: "+str(round(getEstimatedTime(perc, doneTime, 100-perc),2))+"s")

In [21]:
img_list = []

with open("trainKeys.txt") as f:
    img_list += [l[:-1] for l in f.readlines()]
with open("testKeys.txt") as f:
    img_list += [l[:-1] for l in f.readlines()]

len(img_list)

1500

In [22]:
dataPath = "./data/"
total = len(img_list)

In [23]:
final_img_list = []

for k in img_list:
    try:
        imgDone[k]
    except:
        final_img_list.append(k)

print(len(img_list))
print(len(final_img_list))

1500
1500


In [24]:
img_list = final_img_list
total = len(img_list)

In [25]:
f_out = h5py.File(outputname, "a")

t = []

count = 0
start = time.time()
for item in img_list:
    name = dataPath + item
    img = image.load_img(name, target_size=(224, 224))
    x = image.img_to_array(img)
    x = np.expand_dims(x, axis=0)
    x = preprocess_input(x)
    feat = model.predict(x)
    # Flatten(feat)
    # features.append(feat)
    t.append(feat.shape)
    count += 1
    progressBar((count*100)/total,start)
    f_out[item.replace("/", "_")] = np.array(feat)

f_out.close()

[==============================] 100.0%	Est Time: 0.0s


In [30]:
f_out.close()

In [ ]:
t = list(set(t))
t

[(1, 7, 7, 512)]

In [ ]:
str(count) + " images found"

'8035 images found'